In [18]:
import sys, os
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)
print("Added to sys.path:", repo_root)
from fixedincomelib import *
print("Fixed Income Library is loaded.")

Added to sys.path: c:\Users\Wanling Xie\FixedIncomeLib
Fixed Income Library is loaded.


### Build a Yield Curve

In [19]:
### de-serialize a pre-built model
path = 'serialized/yc_model.pickle'
yc_model : Model = qfReadModelFromFile(path)
# rebuild model for special dates
value_date = qfDisplayModelValueDate(yc_model)
data_collection = qfGetDataCollectionFromModel(yc_model)
build_method_collection = qfGetBuildMethodCollection(yc_model)
value_date_last_date = '2026-05-26'
yc_model_last_date = qfCreateModel(value_date_last_date, 'YIELD_CURVE', data_collection, build_method_collection)
value_date_expired = '2026-05-27'
yc_model_expired = qfCreateModel(value_date_expired, 'YIELD_CURVE', data_collection, build_method_collection)

### Build Product RFR Future

In [24]:
effecitve_date = '2026-05-26'
termination_date = '2026-08-26'
term = '3M'
future_convention = 'SOFR-FUTURE-3M'
long_or_short = 'LONG'
amount = 1e4
strike = 0.
prod_rfr_future = qfCreateProductRFRFuture(
    effecitve_date,
    termination_date,
    future_convention,
    long_or_short,
    amount,
    strike)

### Build Valuation Parameter Collection

In [25]:
### create funding index valuation parameters
vp_type = 'FUNDING INDEX PARAMETER'
content = {'Funding Index' : 'SOFR-1B'}
fi_vp = qfCreateValuationParameters(vp_type, content)
vp_collection = qfCreateValuationParametersCollection([fi_vp])

### Test Valuation

In [26]:
qfReloadIndexFixings()
IndexFixingsManager().exists("SOFR-1B")

All fxings are reloaded from file.


True

In [29]:
print("effective:", prod_rfr_future.effective_date_)
print("termination:", prod_rfr_future.termination_date_)

effective: May 26th, 2026
termination: August 26th, 2026


In [27]:
### test pv and cash
report = qfCreateValueReport(yc_model, prod_rfr_future, vp_collection, 'pvdetailed')
display(report.display())
report_last_date = qfCreateValueReport(yc_model_last_date, prod_rfr_future, vp_collection, 'pvdetailed')
display(report_last_date.display())
report_expired = qfCreateValueReport(yc_model_expired, prod_rfr_future, vp_collection, 'pvdetailed')
display(report_expired.display())

,Currency,Type,Value
0,USD,PV,7.673282e+07
1,USD,CASH,0.000000e+00


,Currency,Type,Value
0,USD,PV,0.000000e+00
1,USD,CASH,7.425299e+07


Exception: Cannot find SOFR-1B for date ...

In [7]:
### test risk
rr_report = qfCreateValueReport(yc_model, prod_rfr_future, vp_collection, 'firstorderrisk')
display(rr_report)
rr_report_last_date = qfCreateValueReport(yc_model_last_date, prod_rfr_future, vp_collection, 'firstorderrisk')
display(rr_report_last_date)
rr_report_expired = qfCreateValueReport(yc_model_expired, prod_rfr_future, vp_collection, 'firstorderrisk')
display(rr_report_expired)

[array([0., 0., 0., 0.]), array([0., 0., 0.])]

[array([0., 0., 0., 0.]), array([0., 0., 0.])]

[array([0., 0., 0., 0.]), array([0., 0., 0.])]

In [9]:
### test cf report
cf_report = qfCreateValueReport(yc_model, prod_rfr_future, vp_collection, 'cashflowsreport')
display(cf_report.display())
cf_report_last_date= qfCreateValueReport(yc_model_last_date, prod_rfr_future, vp_collection, 'cashflowsreport')
display(cf_report_last_date.display())
cf_report_expired = qfCreateValueReport(yc_model_expired, prod_rfr_future, vp_collection, 'cashflowsreport')
display(cf_report_expired.display())

,PRODUCT_TYPE,VALUATION_ENGINE_TYPE,LEG_ID,CASHFLOW_ID,PAY_OR_RECEIVE,NOTIONAL,PAY_DATE,FORECASTED_AMOUNT,PV
0,PRODUCT_RFR_FUTURE,ValuationEngineProductRfrFuture,0,0,1.0,2.500000e+09,"May 26th, 2026",7.771025e+07,7.673282e+07


,PRODUCT_TYPE,VALUATION_ENGINE_TYPE,LEG_ID,CASHFLOW_ID,PAY_OR_RECEIVE,NOTIONAL,PAY_DATE,FORECASTED_AMOUNT,PV
0,PRODUCT_RFR_FUTURE,ValuationEngineProductRfrFuture,0,0,1.0,2.500000e+09,"May 26th, 2026",7.425299e+07,7.425299e+07


Exception: Cannot find SOFR-1B for date ...